# 异常的设计

## 嵌套异常处理

try语句是可以嵌套的，python会将try语句放入栈，当异常发生时，python会回到最近进入，具有相符except分句的try语句。然后向上传递到较高的处理程序。一旦异常被捕获，其生命就结束了，控制权并不会跳回所有匹配该异常、相符的try语句，只有第一个匹配的try有机会对它进行处理。、

而当try/finally语句嵌套发生异常时，每个嵌套的finally代码都会依次执行；直到碰到能处理对应异常的except语句

In [1]:
def action2():
    print(1 + [])

def action1():
    try:
        action2()
    except TypeError:
        print('inner try')

In [2]:
# 会在首次except匹配时拦截
try:
    action1()
except TypeError:
    print('outer try')

inner try


In [3]:
def raise1(): raise IndexError
def noraise(): return 
def raise2(): raise SyntaxError

In [4]:
for func in (raise1, noraise, raise2):
    print('<%s>' % func.__name__)
    try:
        try:
            func()
        except IndexError:
            print('caught IndexError')
    finally:
        print('finally run')
    print('...')

<raise1>
caught IndexError
finally run
...
<noraise>
finally run
...
<raise2>
finally run


SyntaxError: None (<string>)

## 异常的习惯用法

### 跳出多重循环

python中的break只能跳出一个最近的外围循环，但是可以利用异常来跳出更多层的循环

In [5]:
class Exitloop(Exception): pass

In [8]:
try:
    while True:
        while True:
            for i in range(10):
                if i > 3:
                    raise Exitloop
                print('loop3 : %s' % i)
            print('loop2')
        print('loop1')
except Exitloop:
    print('continuing')

loop3 : 0
loop3 : 1
loop3 : 2
loop3 : 3
continuing


如果把这里的raise 换成 break, 则会得到一个无限循环，只能退出for循环，然后进入loop2,没有终止条件会一直循环下去

### 异常并不总是错误

所有错误都是异常，但是并非所有异常都是错误。如文件读到最后，或者调用sys.exit()

### 函数可以用raise语句触发状况

用户定义的异常也可以用于触发非错误的状况。例如函数可以写成找到相应数据后引发异常，而不是返回一个状态标志来告知调用者。

### 断开文件或服务器连接

使用finally，可以保证发生异常后，始终能够断开服务

### 使用外层try语句辅助调试

可以利用异常处理程序取代python的默认顶层异常处理行为。通过把整段顶层程序包在一个外层的try语句中，可以捕捉任何程序执行时会引发的异常，从而改变默认的程序终止行为。

In [9]:
try:
    raise IndexError
except:
    import sys
    print(sys.exc_info()[0], sys.exc_info()[1])

<class 'IndexError'> 


### sys.exc_info 

exc_info 允许异常处理程序获取对最近引发的异常的访问。当不想盲目的使用空except分句来捕获每个异常以确定引发了什么的时候，这种方式尤其有用。

如果没有正在处理的异常，这一调用返回一个包含三个None值的元组，否则，将会返回(type, Value, traceback)
- type 正在处理异常的类型
- value 被引发的异常类实例
- traceback 跟踪对象，代表异常最初发生时所调用的栈。

## 异常设计的建议和缺陷

### 应该包装什么

- 经常会失败的一般都应该包装在try语句内
- 应该在 try/finally中实现终止动作，从而保证它们的执行，除非存在可使上下文管理器作为with/as的可选方案
- 把对大型函数的调用包装在单个try语句内，而不是向函数本身中放入零散的try语句内。

### 避免空的except分句和Exception异常
避免涵盖太广。

### 捕获过少

处理程序也不应过于具体。当在try中列出特定的异常时，只能捕捉实力列出的异常。可用层次的类异常解决